# Colorful Image Colorization

*   Abhijoy Sarkar 2019AIM1001
*   Kirtimaan Gogna 2019AIM1014
*   Deepankar Adhikari 2019CSM1004

# Abstract

Given a grayscale photograph as input, this paper attacks
the problem of hallucinating a plausible color version of the photograph.
There is no clear ground truth which the images must conform to. However, we try to produce plausible reproductions of the ground truth for a given grayscale image. Several novelties are introduced that overcome the need for annotation and desaturated results. These include using a custom loss function and class rebalancers to capture the effect of rarer colors. A CNN model architecture predicts a class probability distribution from which a plausible color is selected using an annealed mean technique.

# Introduction

<figure>
  <img src="http://abhijoysarkar.in/image1.png">
  <figcaption><b>Fig. 1</b> Consider this iconic photograph of Yosemite Valley from Ansel Adam. So how would it look like in COLOR? Now, on the face of it, the problem is rather underconstrained. We are looking to go from a 1-dimensional signal into a 3-dimensional signal. However, we know what a plausible image would look like.We know that the sky is probably blue, the mountain is likely brown, and the trees are most definitely green.
</figcaption>
</figure>

Image colorization is the process of taking an input grayscale (black and white) image and then producing an output colorized image that represents the semantic colors and tones of the input (for example, an ocean on a clear sunny day must be plausibly “blue” — it can’t be colored “hot pink” by the model).

Previous methods for image colorization either:

*   Relied on significant human interaction and annotation
*   Produced desaturated colorization

The novel approach used here today instead relies on deep learning. We will utilize a Convolutional Neural Network capable of colorizing black and white images with results that produces plausible results.

The technique we’ll be covering here today is from Zhang et al.’s 2016 ECCV paper, Colorful Image Colorization.

Previous approaches to black and white image colorization relied on manual human annotation and often produced desaturated results that were not “believable” as true colorizations.

Zhang et al. decided to attack the problem of image colorization by using Convolutional Neural Networks to “hallucinate” what an input grayscale image would look like when colorized.

To train the network Zhang et al. started with the ImageNet dataset and converted all images from the RGB color space to the Lab color space.



# Literature Survey

For the task of automatic image colorization, several methods are proposed that showed surprisingly good results on the task using deep networks. The idea was to take a deep network, train on a million images, and directly map from the greyscale image to the color information. 

## Fusion Models for image colorization
There were many previous works that tackled this problem and many of them worked quite well. But there were cases when the results simply didn't make too much sense. For instance the algorithm often didn't guess what color the fur of a dog should be. If we would  give same task to human we would expect better results because the human knows what breed the dog is and what colors are appropriate for that breed. In short the human knows what is actually seen on the image but the algorithm doesn't. It just trains on black and white and colored image pairs and learns how it is usually done without any concept of what is seen on the image. So the idea was to not only get the neural network to colorize the image but also classify what is seen on the image before doing it. For example: if we see a dog in the image it is not likely to be pink, or if we know that we have to deal with a football ground we immediately know to reach out to green trails. Izuka et. al [1] presented a novel fusion based technique that has seperate neural network for classifying the images and one for colorising them. 
<figure>
  <img src="https://i.ibb.co/Qm2LmqF/2.jpg">
  <figcaption><b>Fig. 2</b> Fusion model for image colorization which includes a neural network for image classification as well [1]
</figcaption>
</figure>

The fusion part is of a unification of the informations in the two neural networks made to aggregate all the information to create the output. The results were spectacular. The additional information of what the images were about really made a huge impact on the quality of the results. 
<figure>
  <img src="https://i.ibb.co/2MFDCky/3.jpg">
  <figcaption><b>Fig. 3</b> Results of the Joint End-to-end Learning approach [1]
</figcaption>
</figure>
This was by far not the first work on fusion, similar technique have been used in for recognizing object in videos [2]. The approach used for image colorization [1] was a really creative applicaton of the same train of thought. Some expreiments revealed that temporal coherence is still a problem. This means that every image is colorized separately with no communication. It is a bit like giving the images to colorize one by one to different people with no certain artistic direction. Threfore when the aproach used to render colored videos, the result that as achieved was a flicker render. This problem has been solved under artistic style transfer [3]. 

## Colorization with internet images
Getting input is a heavy task for training models and requires a lot of effort from the user and colorized images would still present unnatural results maybe due to inaccurate transfer of colors. Alex et. al [4] presented a system for colorizing an input greyscale image using colored photos downloaded from internet.
<figure>
  <img src="https://i.ibb.co/VgRv5cg/4.jpg">
  <figcaption><b>Fig. 4</b> : System pipeline of Alex et. al model [4]
</figcaption>
</figure> 
This method required some effort from the user. To colorize a greyscale image the user had to segment the foreground objects using some basic snapping techniques and add text label for each object. The system was modeled to translate the text label to several languages and use the terms to search for photos in several image sharing websites like google, flicker etc. To find the most suitable downloaded images for colorization tast, the system segments their foreground regions and compare them to the greyscale foreground object in terms of shape, intensity, texture and dense sift features. For colorization of background, the same set of images downloaded for coloring foreground and segmentation was assisted by Segras et. al [5]. After finding most suitable foreground regons in downloaded images, each of them were used to colorize the greyscale object using energy minimization framework based on similarity of intensity, texture, dense sift features and spatial features. 
<figure>
  <img src="https://i.ibb.co/JnBqwx7/5.jpg">
  <figcaption><b>Fig. 5</b> : Results: Input grayscale image, resulting colorized image, the last column shows the foreground and background reference examples selected using filtering technique by segras et. al[4]
</figcaption>
</figure> 



# Method
Similar to the RGB color space, the Lab color space has three channels. But unlike the RGB color space, Lab encodes color information differently:

The L channel encodes lightness intensity only
The a channel encodes green-red.
And the b channel encodes blue-yellow
Since the L channel encodes only the intensity, we can use the L channel as our grayscale input to the network.

From there the network must learn to predict the a and b channels. Given the input L channel and the predicted ab channels we can then form our final output image.

The entire (simplified) process can be summarized as:

*    Convert all training images from the RGB color space to the Lab color space.
*    Use the L channel as the input to the network and train the network to predict the ab channels.
*    Combine the input L channel with the predicted ab channels.
*    Convert the Lab image back to RGB.

To produce more plausible black and white image colorizations the authors also utilize a few additional techniques including mean annealing and a specialized loss function for color rebalancing.


## Objective Funcion

For input lightness channel $X\in R^{H\times W \times 1}$ we want to find out $\hat{Y}=F(x)$ where $\hat{Y}$ denotes the ab output channels $Y\in R^{H\times W\times 2}$ with $H,W$ corresponds to image height and width respectively.

$L_2(\hat{Y},Y)=\frac{1}{2}\sum_{h,w}||Y_{h,w} - \hat{Y}_{h,w}||_2^2$

Using the $L_2$ loss given above does not work as we are not trying to predict the ground truth. Our predictions is flexible and may be as plausible as the ground truth even with a different colorization.

Converting the problem to a multinomial classification problem, the ab output space is divided into 10 bins. We combine bins from the two channels to declare our output hence avoiding the real valued predictions. This way there are $Q=313$ values that can be used by the model to color an image. Hence we try to learn a probabilty distribution which is denoted as $\hat{Z}=G(x)$ and $\hat{Z}\in [0,1]^{H\times W\times Q}$.
Hence, our loss function becomes a multinomial cross entropy loss given below:-

$L_{cl}(\hat{Z},Z)=-\sum_{h,w}v(Z_{h,w})\sum_qZ_{h,w,q}\log(\hat{Z}_{h,w,q})$

The $v(Z_{h,w})$ term is explained in following section.

## Class Rebalancing
In a normal euclidean loss function if the object is a multimodal, then the optimal euclidean loss is the mean of the set and the averaging result will favour greysih and desaturated results. To preserve color rarity, color-class rarity based weighting term is used to rebalance the loss.

$v(Z_{h,w})=w_{q^*}$, where $q^*=\underset{x}{\arg\max} Z_{h,w,q}$

$w\propto((1-\lambda)\widetilde{p} + \frac{\lambda}{Q})^{-1}$,

$E[w]=\sum_q\widetilde{p}_qw_q=1$

## Class Probabilities and Point Estimates
$H(Z_{h,w})$ maps the predicted distribution to point estimates Y. The following options were discussed to aggregate on the distribution and each had it's shortcoming: 


*   Mode: Taking mode of the predicted distribution for each pixel gave vibrant images, but at times they were spatially inconsistent
*   Mean: Taking mean produced spatially consistent results but at times they were desaturated.

Therefore the function $H(Z_{h,w})$ was designed to accumulalte the benifits of both the methods and was called annealed-mean.



$H(Z_{h,w})=E[f_T(Z_{h,w})]$,

$f_T(z)=\frac{\exp(\log(z)/T)}{\sum_q\exp(\log(z_q)/T)}$

## Training on CIFAR-100 Dataset

The below code will work if cifar-100-python folder downloaded in your Google drive folder.

First we import the necessary files.

In [0]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
from google.colab import drive
# Uncomment the below line to mount drive
# drive.mount('/gdrive')

Now we define functions to unpickle the CIFAR-100 dataset

In [0]:
def get_proper_images(raw):
    raw_float = np.array(raw, dtype=float) 
    images = raw_float.reshape([-1, 3, 32, 32])
    images = images.transpose([0, 2, 3, 1])
    return images

def onehot_labels(labels):
    return np.eye(100)[labels]

def unpickle(file):
    import pickle
    fo = open(file, 'rb')
    dict = pickle.load(fo,encoding='bytes')
    fo.close()
    return dict

In [0]:
X_train = get_proper_images(unpickle('drive/My Drive/cifar-100-python/train')[b'data'])
Y_train = onehot_labels(unpickle('drive/My Drive/cifar-100-python/train')[b'fine_labels'])
X_test = get_proper_images(unpickle('drive/My Drive/cifar-100-python/test')[b'data'])
Y_test = onehot_labels(unpickle('drive/My Drive/cifar-100-python/test')[b'fine_labels'])

### Data Preprocessing

Seperation of training data and their corresponding labels is done.

In [0]:
X=[]
Y=[]
for image in X_train:
  X.append(rgb2lab(1.0/255*image)[:,:,0])
  Y.append(rgb2lab(1.0/255*image)[:,:,1:])
X=np.array(X)
Y=np.array(Y)
Y /= 128
X=X.reshape(len(X),32,32,1)
Y=Y.reshape(len(Y),32,32,2)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

We are finally ready to build our model

In [0]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

### Fitting the model

We compile our model and see what our model comprises of.

In [0]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')
model.summary()

The model is now fit to the training data and it learns to predict the ab channel from L channel.

In [0]:
model.fit(x=X, 
    y=Y,
    batch_size=1000,
    epochs=10)

We are ready to test it out now. Here we prepare the test set the same way as the training set.

In [0]:
X_T=[]
Y_T=[]
for image in X_test:
  X_T.append(rgb2lab(1.0/255*image)[:,:,0])
  Y_T.append(rgb2lab(1.0/255*image)[:,:,1:])

X_T=np.array(X_T)
Y_T=np.array(Y_T)
Y_test /= 128
X_T=X_T.reshape(len(X_T),32,32,1)
Y_T=Y_T.reshape(len(Y_T),32,32,2)

We take the first data point and try to find out what it looks like when colored. The grayscale version of the image will be stored as `img_gray_version.png` and colorized image is stored in `img_result.png`. We can repeat the same experiment on all the test data in a similar fashion.

In [0]:
result = model.evaluate(X_T, Y_T, batch_size=1)
print(result)
output = model.predict(X_T)
output *= 128
# Output colorizations
cur = np.zeros((32, 32, 3))
cur[:,:,0] = X_T[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

We find plausible results even for this simple network considering the images are `32x32` pixels only. However, we recognize the need for additional layer of complexity for larger images.

## Implementation of Zhang et. al's [6] algorithm

<figure>
  <img src="https://i.ibb.co/D9NrgWC/6.jpg">
  <figcaption><b>Fig. 6</b> : CNN to map from a grayscale input to a distribution over quantized color value outputs using the architecture as show below. [6]
</figcaption>
</figure> 

To see image colorization in action run the below code in an IPython console with the following parameters:-


*   **--image** Location of the image that is to be colorized
*   **--model** The pre-trained neural network stored as a Caffe model
*   **--prototxt** Caffe prototxt file
*   **--points** The path to a NumPy cluster center points file


```
python bw2color_video.py \ 
  --video video/kota \
  --prototxt model/colorization_deploy_v2.prototxt \ 
  --model model/colorization_release_v2.caffemodel \
  --points model/pts_in_hull.npy

```

To see video colorization in action for some pre made video clip run the below code in an IPython console with the following parameters:-

*   **--video** Location of the video that is to be colorized
*   **--model** The pre-trained neural network stored as a Caffe model
*   **--prototxt** Caffe prototxt file
*   **--points** The path to a NumPy cluster center points file

```
python bw2color_video.py \
    --prototxt model/colorization_deploy_v2.prototxt \
    --model model/colorization_release_v2.caffemodel \
    --points model/pts_in_hull.npy \
    --input video/kota.mp4

```

To see video colorization on your webcam in action run the below code in an IPython console with the following parameters:-

```
python bw2color_video.py \
	--prototxt model/colorization_deploy_v2.prototxt \
	--model model/colorization_release_v2.caffemodel \
	--points model/pts_in_hull.npy
```

# Results
In figure 7, we notice that the two images  look very different. But even though red and blue are far apart in ab space, we are just as happy with the red colorization as we are with the blue, and perhaps the red is even better
<figure>
  <img src="https://i.ibb.co/ZhR2HW2/7.jpg">
  <figcaption><b>Fig. 7</b> Result: Plausible rendring from grayscale input
</figcaption>
</figure>

The system does have some interesting failure cases (figure 8 (a,b)). We find that many man-made objects can be multiple colors. The system sometimes has a difficult time deciding which one to go with, leading to this type of tie-dye effect.
<figure>
  <img src="https://i.ibb.co/D7Dmkqj/10.jpg">
  <figcaption><b>Fig. 8 (a)</b> Result: Failure cases
</figcaption>
</figure>
<figure>
  <img src="https://i.ibb.co/grkzyLx/9.jpg">
  <figcaption><b>Fig. 8 (b)</b> Result: Failure cases
</figcaption>
</figure>

Also, we find other curious behaviors and biases. For example (figure 9), when the system sees a dog, it sometimes expects a tongue underneath. Even when there is none, it will just go ahead and hallucinate one for us anyways.
<figure>
  <img src="https://i.ibb.co/QNf9P2b/11.jpg">
  <figcaption><b>Fig. 9</b> Biases
</figcaption>
</figure>
---


[![Colorized Kota Factory Trailer](https://img.youtube.com/vi/QTbm8oeLhKs/0.jpg)](https://youtu.be/QTbm8oeLhKs)
<figcaption><b>Fig. 10</b> Colorized Kota Factory Trailer (click on the image to follow the resulting video)</figcaption>

---





# Conclusion
With the help of these models, we are able to succesfully colorize both black and white images as well as black white videos. Though our model couldn't reproduce the exact ground truth colors but it was able to reproduce very much plausible colors under those given circumstances. It can prdoduce results indistinguisable from real color photos. This method can also be viewed as a pretext for representation learning. Also our model helps in learning the representation that is very useful for object classification, detection and segmentation.



# References

1. Iizuka, Satoshi & Simo-Serra, Edgar & Ishikawa, Hiroshi. (2016). Let there be color!: joint end-to-end learning of global and local image priors for automatic image colorization with simultaneous classification. ACM Transactions on Graphics. 35. 1-11. 10.1145/2897824.2925974. 

2. Karpathy, Andrej & Toderici, George & Shetty, Sanketh & Leung, Thomas & Sukthankar, Rahul & Fei-Fei, Li. (2014). Large-Scale Video Classification with Convolutional Neural Networks. Proceedings of the IEEE Computer Society Conference on Computer Vision and Pattern Recognition. 1725-1732. 10.1109/CVPR.2014.223.

3. Ruder, Manuel & Dosovitskiy, Alexey & Brox, Thomas. (2016). Artistic Style Transfer for Videos. 9796. 26-36. 10.1007/978-3-319-45886-1_3.  

4. Chia, Alex & Zhuo, Shaojie & Gupta, Raj & Tai, Yu-Wing & Cho, David & Tan, Ping & Lin, Stephen. (2011). Semantic Colorization with Internet Images. ACM Trans. Graph.. 30. 156. 10.1145/2070781.2024190. 

5. HAYS, J., AND EFROS, A. A. 2007. Scene completion using
millions of photographs. ACM Trans. Graph (SIGGRAPH 2007)
26, 3.

6. Zhang, Richard & Isola, Phillip & Efros, Alexei. (2016). Colorful Image Colorization. 9907. 649-666. 10.1007/978-3-319-46487-9_40. 